In [3]:
import json
import sys
import random
import gc

sys.path.append("../")

from models.vector_model import vector_model
from tqdm import tqdm
from preprocessors.preprocessor import Preprocessor

import pandas as pd
import numpy as np

import IR_utils

random.seed(0)
np.random.seed(0)

%load_ext autoreload
%autoreload 2

In [4]:
vm = vector_model.from_pretrained("../../models/vm/mdf4000")

test_queries_t1 = IR_utils.load_test_queries(
    "../../data/dataset/queries.jsonl", "../../data/task1_test.tsv"
)[0]

preprocessor = Preprocessor()

In [9]:
for query_id in tqdm(test_queries_t1.keys(), desc="Querying", unit=" queries"):
    query_text = test_queries_t1[query_id]["text"]
    query_index = test_queries_t1[query_id]["id"]

    query_terms = preprocessor.preprocess_query(query_text)
    results = vm.find_similar(query_terms, topn=10)

    #docIDs = [docID for docID, score in results]
    #scores = [score for docID, score in results]
    
    #print(f"{query_index}\t{','.join(docIDs)},{scores[0]}")

Querying:   0%|          | 5/6980 [00:27<10:36:48,  5.48s/ queries]


KeyboardInterrupt: 